# Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import os

# Gather Data

In [2]:
twitter = pd.read_json(r"C:\Programming\Python\Programs_NLP\MLFinalProject\src\data\twitter_data_with_sentiment.json")

In [3]:
twitter.timestamp = pd.to_datetime(twitter.timestamp)

In [202]:
stock_dict = {}
stock_dir = '../src/data/CompleteStockData'

In [203]:
for file in os.listdir('../src/data/CompleteStockData'):
    if file.endswith('.csv'):
        symbol = file.replace('_stocks.csv','')
        stock_dict[symbol] = pd.read_csv(os.sep.join([stock_dir,file]), index_col='date')

In [204]:
for key in stock_dict.keys():
    stock_dict[key].index = pd.to_datetime(stock_dict[key].index)

In [205]:
twitter.head(2)

,id,text,timestamp,source,symbols,company_names,url,verified,sentiment_compound,sentiment_neg,sentiment_pos,sentiment_neu
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,2018-07-18 21:33:26,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True,0.0,0.0,0.0,1.0
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,2018-07-18 22:22:47,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True,0.0,0.0,0.0,1.0


In [206]:
output_df = twitter[twitter.symbols.apply(lambda sym: sym.lower() in stock_dict)][
    ['id', 'timestamp', 'symbols','sentiment_compound', 'sentiment_neg', 'sentiment_pos',
       'sentiment_neu']].copy()

In [236]:
def get_stock_days(sym, ts, output_price='4. close', perc_change=False, close_override=None):
    sym = sym.lower()
    
    if close_override is None:
        before_close = False
        if ts.hour < 16:
            before_close = True
    else:
        before_close = close_override
    
    if before_close:
        location = np.where(stock_dict[sym].index.map(pd.datetime.date)==(ts.date()+pd.Timedelta(days=1)))[0]
    else:
        location = np.where(stock_dict[sym].index.map(pd.datetime.date)==ts.date())[0]
    if len(location)==0:
        return {}
    location = location[0]
    


    if not perc_change:
        res_df = stock_dict[sym].iloc[location-3:location+4].copy()
    else:
        res_df = stock_dict[sym].iloc[location-4:location+4].copy()
        res_df[output_price] = res_df[output_price].pct_change()
        res_df = res_df.iloc[1:]
    return {'t{}'.format(str(idx)):val for idx,val in enumerate(res_df[output_price], -3)}

In [237]:
output_df['price_data_dict'] = output_df.apply(lambda row: get_stock_days(row['symbols'], row['timestamp']), axis=1)

In [238]:
output_df['price_data_dict_perc'] = output_df.apply(lambda row: get_stock_days(row['symbols'], row['timestamp'], perc_change=True), axis=1)

In [239]:
for time_slice in ['t{}'.format(str(idx)) for idx in range(-3,4)]:
    output_df[time_slice] = output_df.price_data_dict.apply(lambda d: d.get(time_slice))

In [240]:
for time_slice in ['t{}_perc'.format(str(idx)) for idx in range(-3,4)]:
    output_df[time_slice] = output_df.price_data_dict_perc.apply(lambda d: d.get(time_slice.replace('_perc','')))

In [247]:
output_df.drop(['price_data_dict','price_data_dict_perc'], axis=1).to_csv(r'C:\Programming\Python\Programs_NLP\MLFinalProject\src\data\twitter_features.csv')

# DOJ Data

In [243]:
doj = pd.read_json(r"C:\Programming\Python\Programs_NLP\MLFinalProject\src\data\doj_data_with_tags_and_industries_and_sentiment.json")

In [244]:
doj.head()

,all_orgs,clean_orgs,components,contents,date,id,industries,organizations,organizations_titles,sectors,tagged_companies,tagged_symbols,title,topics,sentiment_compound,sentiment_neg,sentiment_neu,sentiment_pos
100,"[Stanley Black & Decker Inc., Honeywell Intern...","[stanley black & decker, honeywell internation...",[Environment and Natural Resources Division],Another important step toward cleaning u...,2012-11-19 05:00:00,12-1384,"[Industrial Machinery/Components, Auto Parts:O...","[Stanley Black & Decker Inc., Honeywell Intern...",[],"[Capital Goods, Capital Goods, Energy, Energy,...","[STANLEY BLACK & DECKER INC, HONEYWELL INTERNA...","[swk, hon, xom, hes, bwa, txt, utx]",Agreement Furthers Cleanup of the Quanta Resou...,[],0.9860,0.043,0.865,0.092
10000,"[Goldstar Property Management, , North ...","[goldstar property management, , north star, t...",[Tax Division],"Troy A. Beam of Shippensburg, Pa., was sen...",2012-04-10 04:00:00,12-453,[],"[Goldstar Property Management, , North ...",[],[],[BEAM INC],[beam],Pennsylvania Tax Defier Sentenced to More Than...,[],-0.9502,0.091,0.849,0.060
10007,"[the Bureau of Alcohol, Tobacco and Firearms, ...","[the bureau of alcohol, tobacco and firearms, ...","[Civil Rights Division, Civil Rights - Crimina...","WASHINGTON– Bobby Joe Rogers, 41, of Pensacola...",2012-02-23 05:00:00,12-247,[Major Chemicals],"[the Bureau of Alcohol, Tobacco and Firearms, ...",[American Family Planning Clinic],[Basic Industries],[ROGERS CORP],[rog],"Pensacola, Florida, Man Indicted for Arson at ...",[],-0.9588,0.169,0.734,0.097
10020,"[the Federal Food Drug and Cosmetic Act, Proto...","[the federal food drug and cosmetic act, proto...",[Civil Division],Pfizer Inc. will pay $55 million plus interest...,2012-12-12 05:00:00,12-1488,"[Major Pharmaceuticals, Major Pharmaceuticals]","[the Federal Food Drug and Cosmetic Act, Proto...","[Off-Label Use, IllegallyPromoting Protonix]","[Health Care, Health Care]","[PFIZER INC, WYETH, WYETH, PFIZER INC]","[pfe, wye, wye, pfe]",Pfizer Agrees to Pay $55 Million for Illegally...,[Consumer Protection],0.9931,0.069,0.814,0.117
10021,"[The Department of Justice, H.C.P., Pfizer H.C...","[the department of justice, h.c.p, pfizer h.c....",[Criminal Division],"WASHINGTON – Pfizer H.C.P. Corporation, an i...",2012-08-07 04:00:00,12-980,[Major Pharmaceuticals],"[The Department of Justice, H.C.P., Pfizer H.C...",[Pfizer H.C.P. Corp.],[Health Care],"[PFIZER INC, WYETH, WYETH]","[pfe, wye, wye]",Pfizer H.C.P. Corp. Agrees to Pay $15 Million ...,[],0.9954,0.059,0.806,0.135


# Some regressions

In [53]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [79]:
X = output_df[output_df.sentiment_neg>0].dropna()[['sentiment_neg','t-3_perc','t-2_perc','t-1_perc']]
y = output_df[output_df.sentiment_neg>0].dropna().t3_perc

In [80]:
X = sm.add_constant(X)

In [81]:
model = sm.OLS(y,X)
results = model.fit()

In [82]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                t3_perc   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     12.24
Date:                Thu, 14 Mar 2019   Prob (F-statistic):           6.92e-10
Time:                        19:58:52   Log-Likelihood:                 9480.1
No. Observations:                3615   AIC:                        -1.895e+04
Df Residuals:                    3610   BIC:                        -1.892e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0039      0.001      5.613      0.000       0.003       0.005
sentiment_neg    -0.0108      0.004     -2.742      0.006      -0.019      -0.003
t-3_perc          0.0070      0.020      0.343      0.732      -0.033       0.047
t-2_perc          0.0168      0.019      0.888      0.374      -0.020       0.054
t-1_perc          0.1226      0.020      6.209      0.000       0.084       0.161
==============================================================================
Omnibus:                      786.642   Durbin-Watson:                   1.732
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17447.494
Skew:                           0.471   Prob(JB):                         0.00
Kurtosis:                      13.721   Cond. No.                         70.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
X.shape

(10335, 5)

In [83]:
twitter.sort_values('sentiment_neg', ascending=False)

,id,text,timestamp,source,symbols,company_names,url,verified,sentiment_compound,sentiment_neg,sentiment_pos,sentiment_neu
22121,1019605039902543900,$SRPT slow death 💩,2018-07-18 15:29:19,nobullshytrader,SRPT,Sarepta Therapeutics,None,False,-0.5994,0.661,0.000,0.339
5040,1017782768615469000,Sold $arry 0.35 shit is dead weight,2018-07-13 14:48:16,AllDeltaNoTheta,ARRY,Array BioPharma Inc.,None,False,-0.8360,0.612,0.000,0.388
22030,1019603200138195000,$AAOI vwap reject,2018-07-18 15:22:00,lilnickysmith,AAOI,Applied Optoelectronics,None,False,-0.4019,0.574,0.000,0.426
28123,1019728282487533600,$BTC RANGE CHOP CHOP STOP STOP https://t.co/68...,2018-07-18 23:39:02,cryptoACX,BTC,Bitcoin,None,False,-0.7065,0.540,0.000,0.460
22246,1019608040998297600,Out $MOMO loss,2018-07-18 15:41:15,BigHornTrader,MOMO,Momo Inc.,None,False,-0.3182,0.535,0.000,0.465
22162,1019605734294802400,$TXN CEO resigns over conduct violations. http...,2018-07-18 15:32:05,WilliamWhiteIP,TXN,Texas Instruments Incorporated,https://investorplace.com/2018/07/texas-instru...,False,-0.6908,0.533,0.000,0.467
27686,1019719188640825300,$IBM killing it after hours,2018-07-18 23:02:54,pmomoney,IBM,International Business Machines Corporation,None,False,-0.6597,0.524,0.000,0.476
26939,1019704225612337200,Ohmygod I missed this! no no no! #Timestamp! #...,2018-07-18 22:03:27,gary_weiss,AMD,Advanced Micro Devices,https://twitter.com/sequenceinc/status/1017458...,True,-0.8710,0.521,0.000,0.479
23679,1019638317363720200,$MU Lower highs lower lows. Very bearish! Clos...,2018-07-18 17:41:33,ScottHo30524354,MU,Micron Technology,None,False,-0.6696,0.520,0.000,0.480
24750,1019662852326608900,MGM Resorts denies liability for Las Vegas sho...,2018-07-18 19:19:03,OptionSamurai,MGM,MGM Resorts International,https://buff.ly/2JvqDaU,False,-0.8555,0.509,0.000,0.491


In [84]:
twitter.sort_values('sentiment_pos', ascending=False)

,id,text,timestamp,source,symbols,company_names,url,verified,sentiment_compound,sentiment_neg,sentiment_pos,sentiment_neu
14462,1019240387595112400,$TVIX wow,2018-07-17 15:20:19,tigga117,TVIX,VelocityShares Daily 2x VIX ST ETN,None,False,0.5859,0.000,0.792,0.208
20743,1019575927603384300,$aaoi wow,2018-07-18 13:33:38,patricks01989,AAOI,Applied Optoelectronics,None,False,0.5859,0.000,0.792,0.208
22539,1019613832266813400,$ADBE strong,2018-07-18 16:04:15,rosatodaytrades,ADBE,Adobe Systems Incorporated,None,False,0.5106,0.000,0.767,0.233
21231,1019584878780829700,$FDX nice,2018-07-18 14:09:12,Surftrader79,FDX,FedEx Corporation,None,False,0.4215,0.000,0.737,0.263
20844,1019577449552777200,$TVIX wo wow,2018-07-18 13:39:41,tigga117,TVIX,VelocityShares Daily 2x VIX ST ETN,None,False,0.5859,0.000,0.655,0.345
20764,1019576283175403500,$MOMO strong https://t.co/Wxar96aFD1,2018-07-18 13:35:03,traderstewie,MOMO,Momo Inc.,None,False,0.5106,0.000,0.623,0.377
20927,1019578977445433300,$FDX looks strong,2018-07-18 13:45:45,Surftrader79,FDX,FedEx Corporation,None,False,0.5106,0.000,0.623,0.377
17388,1019402055490781200,$D solid,2018-07-18 02:02:44,ginez29,D,Dominion Energy,None,False,0.1531,0.000,0.615,0.385
17215,1019389097863929900,Nice chart $UTX Thanks for the share https://t...,2018-07-18 01:11:14,GdaytoYou,UTX,United Technologies Corporation,https://twitter.com/bluegrasscap/status/101937...,False,0.7845,0.000,0.612,0.388
21714,1019596311992021000,$CME very nice,2018-07-18 14:54:38,KymLang,CME,CME Group Inc.,None,False,0.4754,0.000,0.607,0.393


# Grouping by symbol + day

In [85]:
twitter.columns

Index(['id', 'text', 'timestamp', 'source', 'symbols', 'company_names', 'url',
       'verified', 'sentiment_compound', 'sentiment_neg', 'sentiment_pos',
       'sentiment_neu'],
      dtype='object')

In [94]:
grouped_twitter = twitter[twitter.symbols.apply(lambda sym: sym.lower() in stock_dict)][
    ['timestamp','sentiment_neg', 'sentiment_pos',
       'sentiment_neu','symbols']].assign(timestamp=twitter.timestamp.apply(pd.datetime.date)).copy().groupby(['symbols','timestamp']).sum().reset_index()

In [95]:
grouped_twitter['price_data_dict_perc'] = grouped_twitter.apply(lambda row: get_stock_days(row['symbols'], 
                                                                                           row['timestamp'], 
                                                                                           perc_change=True,
                                                                                          close_override=False), axis=1)

In [96]:
for time_slice in ['t{}_perc'.format(str(idx)) for idx in range(-3,4)]:
    grouped_twitter[time_slice] = grouped_twitter.price_data_dict_perc.apply(lambda d: d.get(time_slice.replace('_perc','')))

In [98]:
grouped_twitter['sentiment_simple_agg'] = grouped_twitter.sentiment_pos - grouped_twitter.sentiment_neg

In [103]:
X = grouped_twitter.dropna()[['sentiment_neg','t-3_perc','t-2_perc','t-1_perc']]
y = grouped_twitter.dropna().t3_perc

In [104]:
X = sm.add_constant(X)

In [107]:
model = sm.OLS(y,X)
results = model.fit()

In [108]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                t3_perc   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.550
Date:                Thu, 14 Mar 2019   Prob (F-statistic):              0.185
Time:                        20:35:42   Log-Likelihood:                 6286.6
No. Observations:                2174   AIC:                        -1.256e+04
Df Residuals:                    2169   BIC:                        -1.253e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0002      0.000     -0.554      0.580      -0.001       0.000
sentiment_neg    -0.0003      0.001     -0.527      0.598      -0.001       0.001
t-3_perc          0.0052      0.019      0.265      0.791      -0.033       0.043
t-2_perc          0.0057      0.020      0.282      0.778      -0.034       0.045
t-1_perc          0.0520      0.022      2.381      0.017       0.009       0.095
==============================================================================
Omnibus:                      357.407   Durbin-Watson:                   1.688
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6484.992
Skew:                          -0.106   Prob(JB):                         0.00
Kurtosis:                      11.459   Cond. No.                         81.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
grouped_twitter

,symbols,timestamp,sentiment_neg,sentiment_pos,sentiment_neu,price_data_dict_perc,t-3_perc,t-2_perc,t-1_perc,t0_perc,t1_perc,t2_perc,t3_perc,sentiment_simple_agg
0,A,2018-07-16,0.000,0.000,1.000,"{'t-3': 0.009385937002863498, 't-2': -0.001103...",0.009386,-0.001103,-0.010413,0.000000,0.005102,-0.000793,0.018416,0.000
1,A,2018-07-17,0.264,1.917,44.819,"{'t-3': -0.0011032308904649346, 't-2': -0.0104...",-0.001103,-0.010413,0.005102,0.000000,-0.000793,0.018416,-0.004677,1.653
2,A,2018-07-18,0.298,1.672,36.030,"{'t-3': -0.010413379615020535, 't-2': 0.005102...",-0.010413,0.005102,-0.000793,0.000000,0.018416,-0.004677,-0.002036,1.374
3,AABA,2018-07-11,0.747,0.590,11.662,"{'t-3': 0.003317409766454338, 't-2': 0.0047612...",0.003317,0.004761,-0.029090,0.000000,0.011524,-0.001742,0.002417,-0.157
4,AABA,2018-07-12,0.309,1.063,12.629,"{'t-3': 0.004761274963629214, 't-2': -0.029090...",0.004761,-0.029090,0.011524,0.000000,-0.001742,0.002417,0.009376,0.754
5,AABA,2018-07-13,0.000,0.488,3.512,"{'t-3': -0.029090430433065584, 't-2': 0.011523...",-0.029090,0.011524,-0.001742,0.000000,0.002417,0.009376,-0.009289,0.488
6,AABA,2018-07-14,0.318,0.000,0.682,"{'t-3': -0.029090430433065584, 't-2': 0.011523...",-0.029090,0.011524,-0.001742,0.002417,0.009376,-0.009289,-0.017412,-0.318
7,AABA,2018-07-15,1.118,0.817,9.065,"{'t-3': -0.029090430433065584, 't-2': 0.011523...",-0.029090,0.011524,-0.001742,0.002417,0.009376,-0.009289,-0.017412,-0.301
8,AABA,2018-07-16,0.000,0.370,4.630,"{'t-3': 0.011523861171366612, 't-2': -0.001742...",0.011524,-0.001742,0.002417,0.000000,0.009376,-0.009289,-0.017412,0.370
9,AABA,2018-07-17,0.000,0.200,3.800,"{'t-3': -0.001742393780994389, 't-2': 0.002416...",-0.001742,0.002417,0.009376,0.000000,-0.009289,-0.017412,-0.001227,0.200


In [156]:
twitter.timestamp.apply(pd.datetime.date).unique()

array([datetime.date(2018, 7, 18), datetime.date(2018, 7, 9),
       datetime.date(2018, 7, 10), datetime.date(2018, 7, 16),
       datetime.date(2018, 7, 17), datetime.date(2018, 7, 11),
       datetime.date(2018, 7, 19), datetime.date(2018, 2, 23),
       datetime.date(2018, 7, 12), datetime.date(2018, 7, 8),
       datetime.date(2018, 7, 13), datetime.date(2018, 7, 14),
       datetime.date(2018, 7, 15)], dtype=object)